In [9]:
#! /usr/bin/python3

__author__ = "Jun Hu <jh3846@columbia.edu"
__date__ = "Apr 10, 2018"

import logging

logger = logging.getLogger('etl')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter_c = logging.Formatter('[%(asctime)s]-[%(process)d]-[%(thread)d]-[%(name)s]-[%(lineno)s]-[%(levelname)s]: %(message)s')
ch.setFormatter(formatter_c)

logger.addHandler(ch)

import pandas as pd
import time

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df1 = pd.read_pickle("d1.pickle")
df2 = pd.read_pickle("d2.pickle")
df3 = pd.read_pickle("d3.pickle")
df4 = pd.read_pickle("d4.pickle")
df5 = pd.read_pickle("d5.pickle")

In [55]:
df = pd.concat([df1, df2, df3, df4, df5], axis=1)
logger.debug(df.shape)
df.head()

[2018-04-24 17:30:53,052]-[11397]-[140735658623872]-[etl]-[2]-[DEBUG]: (4241, 17)


[2018-04-24 17:30:53,052]-[11397]-[140735658623872]-[etl]-[2]-[DEBUG]: (4241, 17)


,cancer,unigram,bigram,trigram,four_gram,five_gram,six_gram,trigram_skip2,trigram_skip3,unigram_cls,bigram_cls,trigram_cls,four_gram_cls,five_gram_cls,six_gram_cls,trigram_cls_skip2,trigram_cls_skip3
0,THCA,D Y W G Q G R S D D T A V Y F C G D S P P F F ...,DY YW WG GQ QG RS SD DD DT TA AV VY YF FC GD D...,DYW YWG WGQ GQG RSD SDD DDT DTA TAV AVY VYF YF...,DYWG YWGQ WGQG RSDD SDDT DDTA DTAV TAVY AVYF V...,DYWGQ YWGQG RSDDT SDDTA DDTAV DTAVY TAVYF AVYF...,DYWGQG RSDDTA SDDTAV DDTAVY DTAVYF TAVYFC GDSP...,DWQ YGG RDT SDA DTV DAY TVF AYC GSP DPF SPF PF...,RTF SAC GPY DFW SFG PDQ PYG RTF SAC DVA DYR TF...,4 Y W G 5 G 6 3 4 4 3 1 1 Y F 2 G 4 3 P P F F ...,4Y YW WG G5 5G 63 34 44 43 31 11 1Y YF F2 G4 4...,4YW YWG WG5 G5G 634 344 443 431 311 11Y 1YF YF...,4YWG YWG5 WG5G 6344 3443 4431 4311 311Y 11YF 1...,4YWG5 YWG5G 63443 34431 44311 4311Y 311YF 11YF...,4YWG5G 634431 344311 44311Y 4311YF 311YF2 G43P...,4W5 YGG 643 341 431 41Y 31F 1Y2 G3P 4PF 3PF PF...,63F 312 GPY 4FW 3FG P45 PYG 63F 312 411 4Y6 3F...
1,LUSC,Y Y C A R G V V Y Y C A R V G Y Y Y D S S D Y ...,YY YC CA AR RG GV VV YY YC CA AR RV VG GY YY Y...,YYC YCA CAR ARG RGV GVV YYC YCA CAR ARV RVG VG...,YYCA YCAR CARG ARGV RGVV YYCA YCAR CARV ARVG R...,YYCAR YCARG CARGV ARGVV YYCAR YCARV CARVG ARVG...,YYCARG YCARGV CARGVV YYCARV YCARVG CARVGY ARVG...,YCR YAG CRV AGV YCR YAV CRG AVY RGY VYY GYD YY...,YRY YVY CGD AYS RYS VYD GDY YSP YSM YSL CLR AW...,Y Y 2 1 6 G 1 1 Y Y 2 1 6 1 G Y Y Y 4 3 3 4 Y ...,YY Y2 21 16 6G G1 11 YY Y2 21 16 61 1G GY YY Y...,YY2 Y21 216 16G 6G1 G11 YY2 Y21 216 161 61G 1G...,YY21 Y216 216G 16G1 6G11 YY21 Y216 2161 161G 6...,YY216 Y216G 216G1 16G11 YY216 Y2161 2161G 161G...,YY216G Y216G1 216G11 YY2161 Y2161G 2161GY 161G...,Y26 Y1G 261 1G1 Y26 Y11 26G 11Y 6GY 1YY GY4 YY...,Y6Y Y1Y 2G4 1Y3 6Y3 1Y4 G4Y Y3P Y32 Y31 216 1W...
2,OV,R E H S S S S W Y F D Y W G Q G Y Y C A R H E ...,RE EH HS SS SS SS SW WY YF FD DY YW WG GQ QG Y...,REH EHS HSS SSS SSS SSW SWY WYF YFD FDY DYW YW...,REHS EHSS HSSS SSSS SSSW SSWY SWYF WYFD YFDY F...,REHSS EHSSS HSSSS SSSSW SSSWY SSWYF SWYFD WYFD...,REHSSS EHSSSS HSSSSW SSSSWY SSSWYF SSWYFD SWYF...,RHS ESS HSS SSW SSY SWF SYD WFY YDW FYG DWQ YG...,RSY ESF HSD SWY SYW SFG SDQ WYG YRG YHQ CES AD...,6 4 6 3 3 3 3 W Y F 4 Y W G 5 G Y Y 2 1 6 6 4 ...,64 46 63 33 33 33 3W WY YF F4 4Y YW WG G5 5G Y...,646 463 633 333 333 33W 3WY WYF YF4 F4Y 4YW YW...,6463 4633 6333 3333 333W 33WY 3WYF WYF4 YF4Y F...,64633 46333 63333 3333W 333WY 33WYF 3WYF4 WYF4...,646333 463333 63333W 3333WY 333WYF 33WYF4 3WYF...,663 433 633 33W 33Y 3WF 3Y4 WFY Y4W FYG 4W5 YG...,63Y 43F 634 3WY 3YW 3FG 345 WYG Y6G Y65 243 14...
3,COAD,D Y Y F D Y W G Q G Y Y C A R E P N G P A Y Y ...,DY YY YF FD DY YW WG GQ QG YY YC CA AR RE EP P...,DYY YYF YFD FDY DYW YWG WGQ GQG YYC YCA CAR AR...,DYYF YYFD YFDY FDYW DYWG YWGQ WGQG YYCA YCAR C...,DYYFD YYFDY YFDYW FDYWG DYWGQ YWGQG YYCAR YCAR...,DYYFDY YYFDYW YFDYWG FDYWGQ DYWGQG YYCARE YCAR...,DYD YFY YDW FYG DWQ YGG YCR YAE CRP AEN RPG EN...,DDQ YYG YRG YEP AYV YGW YMG YDQ YVG DYK TYE AY...,4 Y Y F 4 Y W G 5 G Y Y 2 1 6 4 P 5 G P 1 Y Y ...,4Y YY YF F4 4Y YW WG G5 5G YY Y2 21 16 64 4P P...,4YY YYF YF4 F4Y 4YW YWG WG5 G5G YY2 Y21 216 16...,4YYF YYF4 YF4Y F4YW 4YWG YWG5 WG5G YY21 Y216 2...,4YYF4 YYF4Y YF4YW F4YWG 4YWG5 YWG5G YY216 Y216...,4YYF4Y YYF4YW YF4YWG F4YWG5 4YWG5G YY2164 Y216...,4Y4 YFY Y4W FYG 4W5 YGG Y26 Y14 26P 145 6PG 45...,445 YYG Y6G Y4P 1Y1 YGW Y2G Y45 Y1G 4Y6 3Y4 1Y...
4,THCA,Y S S G S T L D Y W G Q G Y Y Y G M D V W G Q ...,YS SS SG GS ST TL LD DY YW WG GQ QG YY YY YG G...,YSS SSG SGS GST STL TLD LDY DYW YWG WGQ GQG YY...,YSSG SSGS SGST GSTL STLD TLDY LDYW DYWG YWGQ W...,YSSGS SSGST SGSTL GSTLD STLDY TLDYW LDYWG DYWG...,YSSGST SSGSTL SGSTLD GSTLDY STLDYW TLDYWG LDYW...,YSS SGT SSL GTD SLY TDW LYG DWQ YGG YYM YGD YM...,YSY STW SLG GDQ SYG YMG YDQ YVG EYV TGW NMG YD...,Y 3 3 G 3 3 1 4 Y W G 5 G Y Y Y G 2 4 1 W G 5 ...,Y3 33 3G G3 33 31 14 4Y YW WG G5 5G YY YY YG G...,Y33 33G 3G3 G33 331 314 14Y 4YW YWG WG5 G5G YY...,Y33G 33G3 3G33 G3

In [56]:
start_time = time.time()

df_vec1000 = df.copy()
max_features = 1000

# ou can change the token_pattern parameter from (?u)\\b\\w\\w+\\b (default) to (?u)\\b\\w\\w*\\b; The default matches token that has two or more word characters (in case you are not familiar with regex, + means one or more, so \\w\\w+ matches word with two or more word characters; * on the other hand means zero or more, \\w\\w* will thus match word with one or more characters):
tfidf = TfidfVectorizer(max_features=max_features, stop_words=None, lowercase=False, token_pattern='(?u)\\b\\w\\w*\\b')

for i in range(1, 17):
    start_time1 = time.time()
    df_vec1000.iloc[:, i] = tfidf.fit_transform(df_vec1000.iloc[:, i]).toarray().tolist()
    logger.debug("--- %s seconds ---" % (time.time() - start_time1))
logger.debug("--- %s seconds ---" % (time.time() - start_time))



[2018-04-24 17:32:34,796]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 80.86641097068787 seconds ---


[2018-04-24 17:32:34,796]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 80.86641097068787 seconds ---


[2018-04-24 17:34:01,559]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 86.76229095458984 seconds ---


[2018-04-24 17:34:01,559]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 86.76229095458984 seconds ---


[2018-04-24 17:35:34,378]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 92.8165488243103 seconds ---


...

[2018-04-24 17:43:21,869]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 83.58299684524536 seconds ---


[2018-04-24 17:44:07,806]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 45.931172132492065 seconds ---


[2018-04-24 17:44:07,806]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 45.931172132492065 seconds ---


[2018-04-24 17:45:29,110]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 81.30311894416809 seconds ---


[2018-04-24 17:45:29,110]-[11397]-[140735658623872]-[etl]-[12]-[DEBUG]: --- 81.30311894416809 seconds ---


In [54]:

df["bigram"][0]

[0.0,
 0.0,
 0.01928097028890905,
 0.0,
 0.03749476636982637,
 0.018751803581854312,
 0.0,
 0.0,
 0.03714285596994309,
 0.0,
 0.0,
 0.020123270081409197,
 0.009683804078245243,
 0.01062351679952153,
 0.3237001409615509,
 0.04697908690806154,
 0.009389174085177228,
 0.0563616145152752,
 0.02016139872343284,
 0.009660979349345504,
 0.3976887446099054,
 0.02688127428557722,
 0.0,
 0.0,
 0.0,
 0.010200750053759977,
 0.012670848334878323,
 0.012759655640269239,
 0.01715124516520396,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.018911645534956143,
 0.0,
 0.0,
 0.0,
 0.02847469829626141,
 0.010166978483531501,
 0.06721328137727568,
 0.010550444988204134,
 0.0,
 0.009471443480129172,
 0.01946384029950882,
 0.0,
 0.0,
 0.05610971165718264,
 0.010418373634576414,
 0.009789529501692035,
 0.046494286994730495,
 0.06044127381203916,
 0.0,
 0.04646141680454404,
 0.1208281644407168,
 0.07407586466053111,
 0.0,
 0.08329606639309786,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.009570239593202669,
 0.0,
 0.0,
 0.0

In [11]:
df_vec1000.head()

RecursionError: maximum recursion depth exceeded

TypeError: cannot concatenate object of type "<class 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid